In [6]:
%matplotlib inline

# Credit to joedockrill's code that I used to debug a lot of things
# https://github.com/joedockrill/heroku
# including downloading a different torch file that reduced the size from
# about 700+mb to ~150mb

from IPython.display import clear_output, Image

from matplotlib.pyplot import imshow, axis, show
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import re

from fastai.vision import *
import urllib.request

import warnings

# Ignore warnings for webapp
warnings.filterwarnings('ignore')
# Wasn't able to find a way to just ignore warnings from a specific
# function; ignoring the warnings from clicking classify didn't work
# because it still popped up

# Code that is run and does something
defaults.device = torch.device('cpu')
learn = load_learner(".")

# Notes
# This webapp may be vulnerable to large files that are downloaded from the URLs
# or from direct file uploads. I'm not sure how to handle this.

# Widget for receiving an image link
link1 = widgets.Text(
    description = "Image link: ",
    placeholder = "Type image link here"
)

# Widget that downloads the image when clicked
download1 = widgets.Button(
    description = 'Download!',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Download the image supplied by the link',
)

def dl_img(event):
  with d1_out:
    d1_out.clear_output()
    # Save the image to 'image.png'
    bla = link1
    # Check that input is a string
    if(type(bla.value) is not str):
      print("Error, link must be a string!")
    # Check that input link leads to an image
    elif(re.search('\.((png)|(jpg))', bla.value) is None):
      print("URL must link to a png or jpg file!")
    else:
      try: # Check that there is an image at the link
        urllib.request.urlretrieve(bla.value, 'image.png')
        print('Image downloaded!')
        # Show the image (assumes that there is a file)
        try:
          file = open("./image.png", "rb")
          image1 = file.read()
        except:
          file = open("./default.png", "rb")
          image1 = file.read()
        finally:
          # This isn't showing up for some reason
          img_out = widgets.HBox([widgets.Image(value = image1,
                                      format = 'url')])
      except: # If the image in the link is not found
        print("image not found :(")
download1.on_click(dl_img)

# Widget that classifies the image when clicked
classify1 = widgets.Button(
    description = 'Classify!',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'Classify the image as "draft" or "not draft"',
)

def classify_img(event):
  with c1_out:
    c1_out.clear_output()
    try:
      img = open_image(Path('./image.png'))
    except:
      img = open_image(Path('./default.png'))
      print("Image not found, default loaded instead")
    pred_class,pred_idx,outputs = learn.predict(img)
    print('Class: ' + pred_class.obj)
classify1.on_click(classify_img)

# Widget that shows the downloaded image
img1 = widgets.Button(
    description = 'Show image',
    disabled = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip = 'View the image',
)

def view_img(event):
  with img_out:
    img_out.clear_output(True)
    try:
      img = mpimg.imread('image.png')
    except:
      print("Image not found, default loaded instead")
      img = mpimg.imread('default.png')
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.show()
img1.on_click(view_img)
    

## Output widgets
# Show the link that's being received
def f1(link):
  print(' Link received: {}'.format(link))
out = widgets.interactive_output(f1, {'link': link1})

# Print statement that confirms the download
d1_out = widgets.Output(layout = Layout(margin='auto'))

# Print statement that shows the classification
c1_out = widgets.Output(layout = Layout(margin='auto'))

# Print image
img_out = widgets.Output(layout = Layout(margin = 'auto'))

# Layout of all the widgets
widgets.VBox([
              widgets.HBox([widgets.VBox([link1, download1, classify1]),
              widgets.VBox([out, d1_out, c1_out])]),
              widgets.HBox([img1, img_out])
])